In [214]:
import numpy as np
import math

distances = {
    "PCF": {
        "370": 1157,
        "370EOM": 1205.5,
        "399": 295,
        "935": 445
    },
    "Wavemeter":{
        "370": 760,
        "399": 260
    },
    "TransferCavity":{
        "370": 747.5
    }
}

fiber = {
    "375PM":2.3,
    "780PM":5.3,
    "PCF370":8.4,
    "PCF399":8.4,
    "PCF935":8.6
}


def diameterOutput(focal_length, diameter_input, wavelength): # (mm, um, nm)
    Do = 4*wavelength*(focal_length)/(np.pi*diameter_input)
#     print(f"Beam diameter is {Do}um.")
    return 4*wavelength*(focal_length)/(np.pi*diameter_input) # um

def divergenceDiameter(distance, start_diameter, wavelength): # (mm, um, nm)
    wz = (start_diameter/2*np.sqrt(1+(wavelength*distance/(np.pi*(start_diameter/2)**2))**2))*2
    return wz
    
def couplingEfficiency(MFD, beam_diameter): # (um,um)
    w0 = MFD/2
    wz = beam_diameter/2
    eff_upper = ((w0*wz/(w0**2+wz**2))*math.erf(5/np.sqrt(w0**2*wz**2/(w0**2+wz**2)))**2)
    eff_lower = math.erf(5/np.sqrt(w0**2/2))*math.erf(5/np.sqrt(wz**2/2))/2
    eff = eff_upper/eff_lower
    return eff

def calculate(wavelength, collimator_focal_length, coupler_focal_length,
              distance, start_fiber, end_fiber): # (nm, mm, mm, mm, um, um)
    Do = diameterOutput(collimator_focal_length, start_fiber, wavelength)
    Dd = divergenceDiameter(distance, Do, wavelength)
    Dc = diameterOutput(coupler_focal_length, Dd, wavelength)
    eff = couplingEfficiency(end_fiber, Dc)
#     print(f"Coupling efficiency is {eff}.")

    result = [round(Do,3), round(Dd,3), round(Dc,3), round(eff,3), distance]
    return result

def calculateSet(setup):
    li = [[370,"PCF370"],[370,"PCF370EOM"],[399,"PCF399"],[935,"PCF935"],[370,"Wavemeter370"],[399,"Wavemeter399"],[370,"TransferCavity370"]]
    results = dict()
    for i in li:        
        result = calculate(i[0], setup["collimator"][f"{i[0]}"], setup["coupler"][i[1]], setup["distance"][i[1]],
                           setup["fiber"][f"{i[0]}"],setup["fiber"][i[1]])
        result.append(setup["fiber"][i[1]])
        results[f'{i[1]}'] = result

    for i in results:
        print(f'{i}: \n Do={results[i][0]}um, Dist={results[i][4]}mm,  Dd={results[i][1]}um,  Dc = {results[i][2]}um,  MFD={results[i][5]}um, Eff={results[i][3]}\n')

    return results
    


In [215]:
setup = {
    "fiber":{
        "370": 2.3,
        "399": 2.3,
        "935": 5.3,
        "PCF370": 8.4,
        "PCF370EOM": 8.4,
        "PCF399": 8.4,
        "PCF935": 8.6,
        "Wavemeter370": 2.3,
        "Wavemeter399": 2.3,
        "TransferCavity370": 2.3
    },
    "collimator":{
        "370": 4.02,
        "399": 4.02,
        "935": 11.16
    },
    "coupler":{
        "PCF370": 15,
        "PCF370EOM": 15,
        "PCF399": 15,
        "PCF935": 15,
        "Wavemeter370": 4,
        "Wavemeter399": 4,
        "TransferCavity370": 4
    },
    "distance": {
        "PCF370": 1157,
        "PCF370EOM": 1205.5,
        "PCF399": 295,
        "PCF935": 445,
        "Wavemeter370": 760,
        "Wavemeter399": 260,
        "TransferCavity370": 747.5
    }
}

In [216]:
calculateSet(setup)

PCF370: 
 Do=823.398um, Dist=1157mm,  Dd=1056.496um,  Dc = 6.689um,  MFD=8.4um, Eff=0.981

PCF370EOM: 
 Do=823.398um, Dist=1205.5mm,  Dd=1074.1um,  Dc = 6.579um,  MFD=8.4um, Eff=0.978

PCF399: 
 Do=887.935um, Dist=295mm,  Dd=903.834um,  Dc = 8.431um,  MFD=8.4um, Eff=1.0

PCF935: 
 Do=2506.744um, Dist=445mm,  Dd=2515.637um,  Dc = 7.098um,  MFD=8.6um, Eff=0.987

Wavemeter370: 
 Do=823.398um, Dist=760mm,  Dd=931.16um,  Dc = 2.024um,  MFD=2.3um, Eff=0.992

Wavemeter399: 
 Do=887.935um, Dist=260mm,  Dd=900.309um,  Dc = 2.257um,  MFD=2.3um, Eff=1.0

TransferCavity370: 
 Do=823.398um, Dist=747.5mm,  Dd=927.842um,  Dc = 2.031um,  MFD=2.3um, Eff=0.992



{'PCF370': [823.398, 1056.496, 6.689, 0.981, 1157, 8.4],
 'PCF370EOM': [823.398, 1074.1, 6.579, 0.978, 1205.5, 8.4],
 'PCF399': [887.935, 903.834, 8.431, 1.0, 295, 8.4],
 'PCF935': [2506.744, 2515.637, 7.098, 0.987, 445, 8.6],
 'Wavemeter370': [823.398, 931.16, 2.024, 0.992, 760, 2.3],
 'Wavemeter399': [887.935, 900.309, 2.257, 1.0, 260, 2.3],
 'TransferCavity370': [823.398, 927.842, 2.031, 0.992, 747.5, 2.3]}